In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import numpy as np
import datetime

In [ ]:
data = pd.read_csv('/content/drive/MyDrive/covid-vaccination/data/owid-covid-data2.csv')
elcano = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/Elcano_Royal_Institute-Global_Presence_Requested_Data.xlsx')
wdi = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/wdi.xlsx')
gov_eff = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/gov_eff.xlsx')
soft_30 = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/soft_power_30.xlsx')
G_20 = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/G20_members.xlsx')
gov_response = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/gov+response.xlsx')
exports = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/exports.xlsx')
rule_of_law = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/rule_of_law.xlsx')
nato = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/nato.xlsx')
vaccines = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/definition.xlsx', sheet_name='vaccines')
pop65 = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/pop65.xlsx')
eu = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/eu.xlsx')
health_mil_exp = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/health_mil_exp.xlsx')
health_pc = health_mil_exp[health_mil_exp['Series Name']=='health_exp_pc'].rename(columns={'mrv':'health_exp_pc'}).drop(columns=['Series Name', 'Country Name'])
mil_total = health_mil_exp[health_mil_exp['Series Name']=='military_total'].rename(columns={'mrv':'military_total'}).drop(columns=['Series Name', 'Country Name'])
exp_total = health_mil_exp[health_mil_exp['Series Name']=='exports_total'].rename(columns={'mrv':'exports_total'}).drop(columns=['Series Name', 'Country Name'])
gov_eff_bounds = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/gov_eff_bounds.xlsx')
presence_data = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/presence_data.xlsx')

In [ ]:
data['date'] = pd.to_datetime(data['date'], format="%Y/%m/%d")

In [ ]:
drop_list = ['nan','OWID_KOS','OWID_WRL','HKG','ABW','BES','VGB','COK','CUW','JEY','KIR','FSM']
data.drop(data[data['iso_code'].isin(drop_list)].index, inplace=True)

In [ ]:
data = data[data['location']!='International']

In [ ]:
ends = '2021-05-01'

In [ ]:
first_dates = {}

for i in pd.unique(data['iso_code']):
  if data[(data['iso_code']==i) & (data['date']<=ends)].total_vaccinations.first_valid_index() == None:
    first_dates[i] = 'NaN'
  else:
    first_dates[i] = data.loc[data[(data['iso_code']==i) & (data['date']<=ends)].total_vaccinations.first_valid_index()].date

In [ ]:
last_dates = {}

for i in pd.unique(data['iso_code']):
  if data[(data['iso_code']==i) & (data['date']<=ends)].total_vaccinations.last_valid_index() == None:
    last_dates[i] = 'NaN'
  else:
    last_dates[i] = data.loc[data[(data['iso_code']==i) & (data['date']<=ends)].total_vaccinations.last_valid_index()].date

In [ ]:
first_dates_ = pd.DataFrame(first_dates.items()).rename(columns={0: "iso_code", 1: "first_date"})
last_dates_ = pd.DataFrame(last_dates.items()).rename(columns={0: "iso_code", 1: "last_date"})

In [ ]:
first_dates_.fillna('NaN',inplace=True)
first_dates_['started_vi'] = (first_dates_['first_date'] != 'NaN').astype(int)

In [ ]:
last_dates_.fillna('NaN',inplace=True)
last_dates_['ended_vi'] = (last_dates_['last_date'] != 'NaN').astype(int)

In [ ]:
date_estimation = pd.merge(first_dates_,
                           last_dates_,
                           on='iso_code',
                           how='inner')

dates = date_estimation[date_estimation['started_vi']==1]

In [ ]:
dates['days'] = dates.last_date - dates.first_date
dates = dates[['iso_code','days']]
dates.days = dates.days.astype('timedelta64[D]')

In [ ]:
last_dates_.fillna('NaN',inplace=True)

In [ ]:
end_indices = []

for i in pd.unique(data['iso_code']):

  if last_dates_[last_dates_['iso_code']==i].last_date.values[0] != 'NaN':
    end_indices.append(data[(data['iso_code']==i) & (data['date']==list(pd.to_datetime(last_dates_[last_dates_['iso_code']==i].last_date))[0].strftime('%Y/%m/%d'))].index[0])
  elif len(data[(data['iso_code']==i) & (data['date']==ends)]) > 0:
    end_indices.append(data[(data['iso_code']==i) & (data['date']==ends)].index[0])
  else:
    continue

In [ ]:
data_ = data[data.index.isin(end_indices)]

In [ ]:
variables = ['iso_code','location','total_cases_per_million','total_deaths_per_million','total_vaccinations','total_vaccinations_per_hundred','population', 'population_density','aged_65_older',
         'gdp_per_capita','life_expectancy', 'human_development_index']
data_ = data_[variables]

In [ ]:
data_1 = pd.merge(data_,
                  dates,
                  on='iso_code',
                  how='left')

In [ ]:
data_2 = pd.merge(data_1,
         wdi,
         on='iso_code',
         how='left')

In [ ]:
data_2.drop(columns=['country'], inplace=True)

In [ ]:
data_3 = pd.merge(data_2,
                  gov_eff,
                  on='iso_code',
                  how='left')

In [ ]:
data_3.drop(columns=['country'], inplace=True)

In [ ]:
data_4 = pd.merge(data_3,
                  elcano,
                  left_on='location',
                  right_on='country',
                  how='left')

In [ ]:
data_4.rename(columns={2019:'elcano_index'}, inplace=True)

In [ ]:
data_4.drop(columns=['country'], inplace=True)

In [ ]:
data_5 = pd.merge(data_4,
                  first_dates_,
                  on='iso_code',
                  how='left')

In [ ]:
data_6 = pd.merge(data_5,
         soft_30,
         on='iso_code',
         how='left')

In [ ]:
data_6.drop(columns=['value'], inplace=True)
data_6.rename(columns={'country':'soft_30'}, inplace=True)

In [ ]:
data_6['soft_30'] = data_6['soft_30'].notnull().astype('int')

In [ ]:
data_7 = pd.merge(data_6,
                  G_20,
                  on='iso_code',
                  how='left')

In [ ]:
data_7.rename(columns={'country':'G_20_member'}, inplace=True)

In [ ]:
data_7['G_20_member'] = data_7['G_20_member'].notnull().astype('int')

In [ ]:
data_7.drop(columns=['first_date'], inplace=True)

In [ ]:
data_7 = pd.merge(data_7,
                  health_pc,
                  on='iso_code',
                  how='left')

In [ ]:
data_7 = pd.merge(data_7,
                  mil_total,
                  on='iso_code',
                  how='left')

In [ ]:
data_7 = pd.merge(data_7,
                  exp_total,
                  on='iso_code',
                  how='left')

In [ ]:
data_8 = pd.merge(data_7,
                  exports,
                  on='iso_code',
                  how='left')

In [ ]:
data_9 = pd.merge(data_8,
                  rule_of_law,
                  on='iso_code',
                  how='left')

In [ ]:
data_9 = pd.merge(data_9,
                  pop65,
                  left_on='iso_code',
                  right_on='country_code',
                  how='left')

In [ ]:
data_9 = pd.merge(data_9,
                  gov_eff_bounds,
                  on='iso_code',
                  how='left')

In [ ]:
data_10 = data_9.replace('..', np.nan)

In [ ]:
data_11 = data_10.replace('...', np.nan)

In [ ]:
data_12 = pd.merge(data_11,
                   gov_response,
                   on='iso_code',
                   how='left')

In [ ]:
data_14 = pd.merge(data_12,
                   eu,
                   left_on='iso_code',
                   right_on='code',
                   how='left')

In [ ]:
data_14.rename(columns={'Name':'eu'}, inplace=True)
data_14['eu'] = data_14['eu'].notnull().astype('int')

#data_7.drop(columns=['first_date'], inplace=True)

In [ ]:
data_14['military_pc'] = data_14['military_total'] / data_14['population']
data_14['exports_pc'] = data_14['exports_total'] / data_14['population']

In [ ]:
data_14 = pd.merge(data_14,
                   presence_data,
                   left_on=['iso_code'],
                   right_on=['iso_code'],
                   how='left')

If you want to get the data for eu-weighted robustness: run from here:

In [ ]:
data_14['gdp_pc_euw'] = np.where(data_14['eu']==1, data_14['weight'] * data_14['GDP per capita, PPP (current international $)'], data_14['GDP per capita, PPP (current international $)'])

In [ ]:
data_14['mil_exp_euw'] = np.where(data_14['eu']==1, data_14['weight'] * data_14['Military expenditure (% of GDP)'], data_14['Military expenditure (% of GDP)'])

In [ ]:
data_14['health_exp_euw'] = np.where(data_14['eu']==1, data_14['weight'] * data_14['Current health expenditure (% of GDP)'], data_14['Current health expenditure (% of GDP)'])

In [ ]:
data_14['exports_euw'] = np.where(data_14['eu']==1, data_14['weight'] * data_14['exports'], data_14['exports'])

In [ ]:
data_14['gov_eff_euw'] = np.where(data_14['eu']==1, data_14['weight'] * data_14['gov_eff'], data_14['gov_eff'])

... to here

In [ ]:
data_14['cty_out'] = np.where((data_14['location']=='China') | (data_14['location']=='Russia'),0,1)

If you want the robustness to control for over 20% vaccinated population, run this:

In [ ]:
data_14['over_20'] = np.where(data_14['total_vaccinations'] / data_14['population'] > 0.2,1,0)

Run this if you chose to get data for eu-weighted robustness:

In [ ]:
euw_cty = {'iso_code':'EU',
           'health_exp_euw':np.sum(data_14[data_14['eu']==1]['health_exp_euw']),
           'mil_exp_euw':np.sum(data_14[data_14['eu']==1]['mil_exp_euw']),
           'gdp_pc_euw':np.sum(data_14[data_14['eu']==1]['gdp_pc_euw']),
           'exports_euw':np.sum(data_14[data_14['eu']==1]['exports_euw']),
           'gov_eff_euw':np.sum(data_14[data_14['eu']==1]['gov_eff_euw']),
           'eu':0,
           'total_cases_per_million':np.array(data_14[data_14['iso_code']=='OWID_EUN']['total_cases_per_million'])[0],
           'total_deaths_per_million':np.array(data_14[data_14['iso_code']=='OWID_EUN']['total_deaths_per_million'])[0],
           'total_vaccinations':np.array(data_14[data_14['iso_code']=='OWID_EUN']['total_vaccinations'])[0],
           'total_vaccinations_per_hundred':np.array(data_14[data_14['iso_code']=='OWID_EUN']['total_vaccinations_per_hundred'])[0],
           'days':np.array(data_14[data_14['iso_code']=='OWID_EUN']['days'])[0],
           'soft_30':1}

data_14 = data_14.append(euw_cty, ignore_index=True)

In [ ]:
drop_list = ['OWID_AFR','OWID_ASI','OWID_EUR','OWID_NAM','OWID_CYN','OWID_OCE','OWID_SAM','OWID_EUN']
data_14.drop(data_14[data_14['iso_code'].isin(drop_list)].index, inplace=True)

Run this if you chose to get data for eu-weighted robustness:

In [ ]:
data_14 = data_14[data_14['eu']==0]

Export your dataset:

In [ ]:
data_14.to_excel('/content/drive/MyDrive/covid-vaccination/data/estimation_data_0501_euw.xlsx')

From here onwards, figures and descriptive stats: 

In [ ]:
data_12 = data_14

In [ ]:
corr_data = pd.DataFrame()

corr_data['lvaccinations'] = np.log(data_12['total_vaccinations_per_hundred'])
corr_data['lcases'] = np.log(data_12['total_cases_per_million'])
corr_data['days'] = data_12['days']
corr_data['lgov_response'] = np.log(data_12['gov_response'])
corr_data['gov_eff'] = data_12['gov_eff']
corr_data['soft_presence'] = data_12['soft_presence']
corr_data['trade'] = data_12['Trade (% of GDP)']
corr_data['lhealth'] = np.log(data_12['health_exp_pc'])
corr_data['lgdp_ppp_pc'] = np.log(data_12['GDP per capita, PPP (current international $)'])
corr_data['lmilitary'] = np.log(data_12['military_pc'])
corr_data['lexports'] = np.log(data_12['exports_pc'])
corr_data['lpop65'] = np.log(data_12['aged_65_older'])
corr_data['life_expectancy'] = np.log(data_12['life_expectancy'])
corr_data['GINI'] = np.log(data_12['GINI'])
corr_data['physicians'] = np.log(data_12['Physicians (per 1,000 people)'])

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
f, ax = plt.subplots(figsize=(16, 10))

corr = corr_data.corr()
ax = sns.heatmap(
    corr, 
    vmin=-1, vmax=1, center=0,
    cmap=sns.diverging_palette(20, 220, n=200),
    square=True,
    annot=True,
    annot_kws={"fontsize":12}  
)
ax.set_xticklabels(
    ax.get_xticklabels(),
    rotation=45,
    horizontalalignment='right'
);

#sns.set(font_scale=10)
plt.savefig("corr_map.png")

Summary...

In [ ]:
summary_data = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/estimation_data_0401.xlsx')

In [ ]:
summary_data['gdp'] = summary_data['GDP per capita, PPP (current international $)'] * summary_data['population']

In [ ]:
summary_data['cases'] = np.log(summary_data['cases'])
summary_data['gov_respose'] = np.log(summary_data['gov_response'])
summary_data['exports'] = np.log(summary_data['exports'])
summary_data['health_exp'] = np.log(summary_data['health_exp'])
summary_data['military'] = np.log(summary_data['military'])
summary_data['pop65'] = np.log(summary_data['pop65'])
summary_data['vi_per_hundred'] = np.log(summary_data['vi_per_hundred'])
summary_data['gdp_ppp_pc'] = np.log(summary_data['gdp_ppp_pc'])

In [ ]:
summary_data.columns

In [ ]:
summary_data = pd.read_excel('/content/drive/MyDrive/covid-vaccination/data/estimation_data_0401.xlsx')

In [ ]:
a = summary_data[['iso_code', 'location','total_cases_per_million',
              'gov_eff', 'soft_presence', 'gov_response','health_exp_pc','military_pc',
              'GDP per capita, PPP (current international $)','pop65','exports_pc','started_vi']].dropna()

In [ ]:
iso_codes = list(a[a['started_vi']==1].iso_code)

In [ ]:
pd.DataFrame(a.groupby('started_vi').agg({'total_cases_per_million':[np.mean, np.std],
                                        'gov_response':[np.mean, np.std],
                                        'exports_pc':[np.mean, np.std],
                                        'health_exp_pc':[np.mean, np.std],
                                        'military_pc':[np.mean, np.std], 
                                        'gov_eff':[np.mean, np.std],
                                        #'total_vaccinations_per_hundred':[np.mean, np.std],
                                        'pop65':[np.mean, np.std],
                                        #'days':[np.mean, np.std],
                                        'GDP per capita, PPP (current international $)':[np.mean, np.std],
                                        'soft_presence':[np.mean, np.std]}).T)#.to_excel('/content/drive/MyDrive/covid-vaccination/results/summary_1208.xlsx')

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

In [ ]:
lreg_data = summary_data[['soft_presence','started_vi']].dropna()

In [ ]:
lreg_data['soft_presence'] = np.log(lreg_data['soft_presence'])

In [ ]:
sns.set_theme(style="darkgrid")

ax = sns.scatterplot(data=lreg_data, x="soft_presence", y="started_vi", s=100)
g = sns.regplot(data=lreg_data, x="soft_presence", y="started_vi", scatter=False, ax=ax, logistic=True)

g.figure.set_size_inches(10,7)


plt.xlabel('$GDP$ $(PPP)$ $per$ $capita$ $(log)$', fontsize=15)
plt.ylabel('$Started$ $vaccination$', fontsize=15)
plt.savefig("started_gdp2.png")

In [ ]:
fig_data = summary_data[['started_vi','soft_presence']].dropna()

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
fig_data_ = fig_data[fig_data['soft_presence']<200]

print(len(fig_data))
print(len(fig_data_))

In [ ]:
import seaborn as sns

ax = sns.boxplot(x="started_vi", y='soft_presence',
                 data=fig_data_)

plt.xlabel('$Started$ $vaccination$', fontsize=15)
plt.ylabel('$Soft$ $presence$', fontsize=15)
plt.xticks([0, 1], ['$No$', '$Yes$'], fontsize=12)
ax.figure.set_size_inches(10,7)
plt.savefig("started_soft.png")

In [ ]:
fig_data['started_vi'].value_counts()

In [ ]:
summary_data['lvaccinations'] = summary_data['total_vaccinations_per_hundred']

In [ ]:
#summary_data = summary_data[summary_data['lvaccinations']<=30]

In [ ]:
fig_data = summary_data[summary_data['iso_code'].isin(iso_codes)]

In [ ]:
fig_data = fig_data[['iso_code','lvaccinations','gov_eff']].dropna()

In [ ]:
pd.DataFrame(pd.merge(fig_data,
         summary_data,
         on='iso_code',
         how='inner').location).to_excel('/content/drive/MyDrive/covid-vaccination/results/started_countries.xlsx')

In [ ]:
fig_data['lvaccinations'] = np.where(fig_data['lvaccinations']==0, 1, fig_data['lvaccinations'])

In [ ]:
fig_data['lvaccinations'] = np.where(fig_data['lvaccinations']==-np.inf, 0, fig_data['lvaccinations'])

In [ ]:
x_data = fig_data.gov_eff
y_data = fig_data.lvaccinations

from scipy import stats

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp

In [ ]:
def regress(x, y):
    """Return a tuple of predicted y values and parameters for linear regression."""
    p = stats.linregress(x, y)
    b1, b0, r, p_val, stderr = p
    y_pred = sp.polyval([b1, b0], x)
    return y_pred, p

x, y = x_data, y_data                            # data, non-transformed
y_pred, _ = regress(x, np.log(y))                 # transformed input             


sns.set_theme(style="darkgrid")

plt.figure(figsize=(10,6))
sns.set_context("paper", font_scale=1.5, rc={"lines.linewidth": 2.5})
plt.plot(x, y, "o", label="Data", markersize=10)
plt.plot(x, np.exp(y_pred), "--", label="Fitted", color='r') # transformed output
plt.xlabel("$Government$ $effectiveness$")
plt.ylabel("$Vaccinations$ $p.h.p.$")
plt.semilogy()
plt.legend()
plt.savefig("vacc_gov_eff.png")